### Use ``Open Route Service API`` to retrieve distance matrix (Single Call version)
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 
#### The following processes 2022 separately since no place data is available for 2022, by assumption we use the most recent 2021 instead

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/result_place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(75786, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2021]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude']).reset_index(drop=True)
print(place_df.shape)
place_df.head(10)

(9283, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,WEST MELBOURNE MARKET,-37.81040,144.92237,2021,market,206041127
1,LILYDALE MARKET,-37.76040,145.34931,2021,market,211051278
2,PRESTON MARKET,-37.73790,145.00264,2021,market,209021428
3,PRAHRAN MARKET,-37.84615,144.99235,2021,market,206061516
4,VICTORIA MARKET,-37.80651,144.95653,2021,market,206041504
5,SOUTH MELBOURNE MARKET,-37.83263,144.95681,2021,market,206051512
6,CROYDON MARKET,-37.79374,145.27820,2021,market,211031451
7,SWAN HILL HOSPITAL,-35.34040,143.55626,2021,hospital,215031404
8,CLUNES HOSPITAL,-37.30179,143.78265,2021,hospital,201021010
9,MALDON HOSPITAL,-36.99270,144.06400,2021,hospital,202021028


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2_classified/2022_property_with_SA2_classified.csv')[PROPERTY_COLS]
# 2022 example
print(property_df.shape)
property_df.head(10)

(73107, 12)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,postcode,year,month,residence_type,SA2_CODE
0,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070
1,"17 PARK STREET, CHILTERN",-36.148025,146.608721,3.0,2.0,0,350,3683,2022,8,House,204031070
2,"3/1 OXFORD STREET, CHILTERN",-36.150694,146.606488,2.0,1.0,1,300,3683,2022,4,Apartment,204031070
3,"2/20 NICKLESS STREET, CHILTERN",-36.154038,146.602977,2.0,1.0,2,350,3683,2022,4,Apartment,204031070
4,"1/3 BURKE STREET, CHILTERN",-36.153004,146.607465,2.0,1.0,0,240,3683,2022,2,Apartment,204031070
5,"590 KENNEDYS LANE, CHINTIN",-37.414852,144.825852,4.0,2.0,7,850,3756,2022,8,House,210021235
6,"4 SHERWOOD ROAD, CHIRNSIDE PARK",-37.747341,145.332546,4.0,2.0,2,680,3116,2022,8,House,211051275
7,"4 SHERWOOD ROAD, CHIRNSIDE PARK",-37.747341,145.332546,4.0,2.0,2,680,3116,2022,9,House,211051275
8,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,480,3116,2022,8,House,211051275
9,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,480,3116,2022,9,House,211051275


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(f"shape = {merged_df.shape}")
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

shape = (1351949, 17)
Distinct sa2 codes = 502


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN POLICE STATION,-36.14815,146.61057,police
1,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN PRIMARY SCHOOL,-36.14707,146.60660,primary
2,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,MIDDLE INDIGO PRIMARY SCHOOL,-36.15601,146.72484,primary
3,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,BARNAWARTHA PRIMARY SCHOOL,-36.10131,146.67589,primary
4,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,ST JOSEPHS SCHOOL,-36.14598,146.60707,primary
5,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN,-36.15566,146.61138,station
6,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,BARNAWARTHA,-36.10280,146.67959,station
7,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN RAILWAY STATION,-36.15540,146.61181,station
8,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN VALLEY RECREATION RESERVE,-36.14293,146.54548,park
9,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN PARK,-36.12429,146.55681,park


### Create the directory to store merged distance data

In [6]:
if not os.path.exists('../../data/featured'):
    os.makedirs('../../data/featured')

#### Specify the ``client`` and ``year`` here, as well as adding a mode of testing/saving

In [7]:
client = ors.Client('5b3ce3597851110001cf6248ce6c95ac96814219a4c3a7741f323b73') # Phikho-caz's api key 2500, back up for at least 4 full year

In [8]:
added_distance_merged_df = add_distance_time(merged_df, 2022, client, 'saving') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 204031070
Subset size = 170, Places count = 10, Property count = 17
To Place Distance Grand List, Normal Branch, length = 170
Executing SA2 Code 210021235
Subset size = 1079, Places count = 13, Property count = 83
To Place Distance Grand List, Normal Branch, length = 1249
Executing SA2 Code 211051275
Subset size = 186, Places count = 3, Property count = 62
To Place Distance Grand List, Normal Branch, length = 1435
Executing SA2 Code 211051278
Subset size = 3640, Places count = 20, Property count = 182
To Place Distance Grand List, Slice Branch, length = 3255
To Place Distance Grand List, Slice Branch, length = 5075
Executing SA2 Code 211051276
Subset size = 2325, Places count = 25, Property count = 93
To Place Distance Grand List, Normal Branch, length = 7400
Executing SA2 Code 205041094
Subset size = 4284, Places count = 42, Property count = 102
To Place Distance Grand List, Slice Branch, length = 9542
To Place Distance Grand List, Slice Branch, length = 11684
Execu

/home/phikhocaz/.local/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


To Place Distance Grand List, Slice Branch, length = 944877
To Place Distance Grand List, Slice Branch, length = 947712
Executing SA2 Code 213041571
Subset size = 385, Places count = 7, Property count = 55
To Place Distance Grand List, Normal Branch, length = 948097
Executing SA2 Code 206011496
Subset size = 4900, Places count = 28, Property count = 175
To Place Distance Grand List, Slice Branch, length = 950533
To Place Distance Grand List, Slice Branch, length = 952997
Executing SA2 Code 210011533
Subset size = 770, Places count = 7, Property count = 110
To Place Distance Grand List, Normal Branch, length = 953767
Executing SA2 Code 205021081
Subset size = 4224, Places count = 32, Property count = 132
To Place Distance Grand List, Slice Branch, length = 955879
To Place Distance Grand List, Slice Branch, length = 957991
Executing SA2 Code 208021177
Subset size = 6111, Places count = 21, Property count = 291
To Place Distance Grand List, Slice Branch, length = 961036
To Place Distance 

/home/phikhocaz/.local/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/phikhocaz/.local/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


To Place Distance Grand List, Normal Branch, length = 1344255
Executing SA2 Code 211031452
Subset size = 252, Places count = 4, Property count = 63
To Place Distance Grand List, Normal Branch, length = 1344507
Executing SA2 Code 210031536
Subset size = 1144, Places count = 11, Property count = 104
To Place Distance Grand List, Normal Branch, length = 1345651
Executing SA2 Code 202031032
Subset size = 189, Places count = 27, Property count = 7
To Place Distance Grand List, Normal Branch, length = 1345840
Executing SA2 Code 215011386
Subset size = 2323, Places count = 23, Property count = 101
To Place Distance Grand List, Normal Branch, length = 1348163
Executing SA2 Code 207011146
Subset size = 2336, Places count = 16, Property count = 146
To Place Distance Grand List, Normal Branch, length = 1350499
Executing SA2 Code 216021414
Subset size = 702, Places count = 13, Property count = 54
To Place Distance Grand List, Normal Branch, length = 1351201
Executing SA2 Code 215031401
Subset size

In [9]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,postcode,year,month,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.61057,police,1114.17,1.11417,150.73,2.512167,281108.72,281.10872,11276.89,187.948167
1,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.60660,primary,1112.89,1.11289,212.67,3.544500,281108.72,281.10872,11276.89,187.948167
2,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.72484,primary,12138.96,12.13896,864.34,14.405667,281108.72,281.10872,11276.89,187.948167
3,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.67589,primary,10137.14,10.13714,574.50,9.575000,281108.72,281.10872,11276.89,187.948167
4,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.60707,primary,1483.77,1.48377,190.73,3.178833,281108.72,281.10872,11276.89,187.948167
5,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.61138,station,855.06,0.85506,168.58,2.809667,281108.72,281.10872,11276.89,187.948167
6,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.67959,station,12039.37,12.03937,788.34,13.139000,281108.72,281.10872,11276.89,187.948167
7,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.61181,station,894.87,0.89487,170.83,2.847167,281108.72,281.10872,11276.89,187.948167
8,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.54548,park,6891.41,6.89141,596.61,9.943500,281108.72,281.10872,11276.89,187.948167
9,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,...,146.55681,park,7350.42,7.35042,587.63,9.793833,281108.72,281.10872,11276.89,187.948167
